# Orbit response 

In [ ]:
import sys
import os
from os.path import join
import collections
from datetime import datetime
import importlib
import h5py
import numpy as np
from pprint import pprint
from tqdm import tqdm
from matplotlib import pyplot as plt
import proplot as pplt

sys.path.append('..')
from tools.data import H5Reader
from tools.plotting import linear_fit

In [ ]:
pplt.rc['grid'] = False
pplt.rc['cmap.discrete'] = False
pplt.rc['cmap.sequential'] = 'viridis'

## Setup 

In [ ]:
folder = '../Diagnostics/Data/Measurements/orbit_response/2022-05-13/'
filenames = os.listdir(folder)
filenames

In [ ]:
class DataFile(H5Reader):
    def __init__(self, filename, folder='.'):
        super().__init__(filename, folder)
        self.quad, self.screen, self.cam = self.filename.split('-')[-1].split('.h5')[0].split('_')
        self.screen_int = self.screen[-2:]
        self.quad_int = self.quad[-2:]
        self.cam_int = self.cam[-2:]

Create nested dictionary of data files. 

In [ ]:
dfiles = dict()
for filename in filenames:
    df = DataFile(filename, folder)
    if df.screen not in dfiles:        
        dfiles[df.screen] = dict()
    dfiles[df.screen][df.quad] = df

In [ ]:
screens = sorted(dfiles)
print('screens:', screens)
print('dfiles:')
pprint(dfiles)

## Analysis

In [ ]:
screen = 'VS14'
quads = sorted(dfiles[screen].keys(), key=lambda quad: int(quad[-2:]))
print('quads:', quads)

In [ ]:
istart = 1  # Ignore the first data point.

In [ ]:
fig, axes = pplt.subplots(ncols=7, nrows=len(screens), figwidth=10.0, sharey=False)
for row, screen in enumerate(screens):
    for col, (ax, quad) in enumerate(zip(axes[row, :], quads)):
        df = dfiles[screen][quad]
        data = df.data
        name_rb = quad + '_CurrentRb'
        name_set = quad + '_CurrentSet'
        x = np.arange(istart, len(df.data))
        ax.plot(x, data[istart:, name_rb], color='black', label='CurrentRb', **kws)
        ax.plot(x, data[istart:, name_set], color='black', alpha=0.2, label='CurrentSet', **kws)        
axes.format(xlabel='Step', leftlabels=screens, toplabels=quads, suptitle='Readback vs. Setpoint')
plt.show()

In [ ]:
def get_mean(profile, x=None):
    if x is None:
        x = np.arange(len(profile))
    return np.average(x, weights=profile)

In [ ]:
rb = 'Set'  # {'Set', 'Rb'}
# Profile cropping -- tune these to get good mean calculations 
cuts = {
    'X': {
        'VS14': (100, 1),
        'VS22': (0, 200),
        'VS30': (150, 120),
        'VS34': (100, 100),
    },
    'Y': {
        'VS30': (100, 100),
    }
}

In [ ]:
for dim in ['X', 'Y']:
    fig, axes = pplt.subplots(ncols=7, nrows=len(screens), figwidth=10.0)
    for row, screen in enumerate(screens):
        for col, (ax, quad) in enumerate(zip(axes[row, :], quads)):

            df = dfiles[screen][quad]
            data = df.data

            profiles = data[istart:, df.cam + '_Profile' + dim]
            
            offsets = 1e7 * np.arange(profiles.shape[0])    
            x = np.arange(profiles.shape[1])

            l, r = (0, 0)
            if screen in cuts[dim]:
                l, r = cuts[dim][screen]
                x = x[l:-r]
                profiles = profiles[:, l:-r]
            means = np.array([get_mean(profile, x=x) for profile in profiles])

            ax2 = ax.altx(xticks=[])
            ax2.plot(profiles.T + offsets, x, color='black', alpha=0.1, lw=1.0)
            
            _profiles = np.zeros(profiles.shape)
            for i, offset in enumerate(offsets):
                _profiles[i, :] += offset
                for j in range(_profiles.shape[1]):
                    if j == int(means[i] - l):
                        _profiles[i, j] += profiles[i, j]
                    else:
                        _profiles[i, j] = None
            ax2.plot(_profiles.T, x, color='pink6', lw=0, marker='.', ms=1)

            current_rb_pv = quad + '_Current' + rb
            x = data[current_rb_pv]
            x = x[istart:]
            x_mean = np.mean(x)
            x_frac_dev = (x - x_mean) / x_mean
            
#             ax.plot(x_frac_dev, np.zeros(len(x_frac_dev)), lw=0)
#             ax.plot(x_frac_dev, means, marker='.', lw=0, ms=1, color='pink6', zorder=9999, alpha=1.0)

    axes.format(
        ylabel='Position [pixel]', 
        xlabel=f'Fractional quad change ({rb})',
        leftlabels=[f'{screen}' for screen in screens],
        toplabels=quads,
        xlim=(-0.1, 0.1),
    )
    n_pixels = {'X': 612, 'Y': 512}[dim]
    
    for i, ax in enumerate(axes[:, 0]):
        ymin, ymax = ax.get_ylim()
        df = -0.1 * np.abs(ymax - ymin)
        axes[i, :].format(ylim=(ymin - df, ymax + df))
    
    plt.savefig(f'_output/waterfall{dim}')
    plt.show()

In [ ]:
slopes = np.zeros((2, len(screens), len(quads)))

In [ ]:
rb = 'Rb'
colors = pplt.Cycle('colorblind').by_key()['color']
line_kws = dict(alpha=0.2)
scatter_kws = dict(marker='.', alpha=0.9, ec='None')

fig, axes = pplt.subplots(ncols=7, nrows=len(screens), figwidth=10.0)
for row, screen in enumerate(screens):
    for col, (ax, quad) in enumerate(zip(axes[row, :], quads)):
        df = dfiles[screen][quad]
        data = df.data
        current_rb_pv = quad + '_Current' + rb
        x = data[current_rb_pv]
        x = x[istart:]
        x_mean = np.mean(x)
        x_frac_dev = (x - x_mean) / x_mean
        for i, (_ax, dim, yloc, halign) in enumerate(zip([ax, ax], ['X', 'Y'], [0.98, 0.88], ['left', 'right'])):
            profiles = data[f'{df.cam}_Profile' + dim]
            profiles = profiles[istart:]
            pos = np.arange(profiles.shape[1])
            if screen in cuts[dim]:
                l, r = cuts[dim][screen]
                pos = pos[l:-r]
                profiles = profiles[:, l:-r]
            means = np.array([get_mean(profile, x=pos) for profile in profiles])
            yfit, slope, intercept = linear_fit(x, means)
            slopes[i, row, col] = slope
            _ax.plot(x_frac_dev, yfit, color=colors[i], **line_kws)
            _ax.scatter(x_frac_dev, means, c=colors[i], label=dim.lower(), **scatter_kws)

# Pad limits.
frac_pad_x = 0.05
frac_pad_y = 0.0
for row, ax in enumerate(axes[:, 0]):
    vmin, vmax = ax.get_ylim()
    pad = frac_pad_y * abs(vmax - vmin)
    axes[row, :].format(ylim=(vmin - pad, vmax + pad))
for col, ax in enumerate(axes[-1, :]):
    vmin, vmax = ax.get_xlim()
    if vmin > vmax:
        vmax, vmin = vmin, vmax
    pad = frac_pad_x * abs(vmax - vmin)
    axes[:, col].format(xlim=(vmin - pad, vmax + pad))

axes.format(
    ylabel='Position [pixel]', 
    xlabel=f'Fractional quad change ({rb})',
    leftlabels=[f'{screen}' for screen in screens],
    toplabels=quads,
)
ylims = [ax.get_ylim() for ax in axes[:, 0]]
plt.savefig(f'_output/orbit_response_{rb}')
plt.show()

In [ ]:
titles = ['Horizontal', 'Vertical']
x = quads
y = screens
for i in range(2):
    fig, ax = pplt.subplots(figwidth=4.5, aspect=2)
    H = slopes[i, :, :].copy()
    vmax = np.max(np.abs(H))
    vmin = -vmax
    ax.pcolormesh(x, y, H, colorbar=True, vmin=vmin, vmax=vmax, 
                  colorbar_kw=dict(label='dx / dQ'), cmap='Vlag')
    ax.format(title=titles[i], xlabel='Quad', ylabel='Screen',
              xlabel_kw=dict(fontweight='bold'),
              ylabel_kw=dict(fontweight='bold'))
    plt.savefig(f'_output/slopes_{titles[i]}')
    plt.show()